In [191]:
import requests

url = 'http://www.pitchfork.com/robots.txt'
response  = requests.get(url)
print(response.text)

User-agent: *
Allow: /
Disallow: /search/
Disallow: /search

Sitemap: https://pitchfork.com/sitemap.xml
Sitemap: https://pitchfork.com/branded-sitemap.xml
Sitemap: https://pitchfork.com/feed/google-latest-news/sitemap-google-news
Sitemap: https://pitchfork.com/feed/sitemap


In [192]:
test = requests.get("https://pitchfork.com/reviews/albums/?page=2")
print(test)

<Response [200]>


In [193]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup

## allows us to use reg expressions to search fields
import re

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC

import random
import time
import csv
import os

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

# user agent
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')


Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0


### II. Selenium to scrape album contents

In [194]:
# -*- coding: utf-8 -*-
"""
Created on Sun Dec  3 20:13:09 2017
@author: Evan
"""
import requests 
from bs4 import BeautifulSoup
import pandas as pd

# driver = webdriver.Chrome(chromedriver,chrome_options=options)

# #_url = "https://www.discogs.com/sell/list?sort=price%2Cdesc&limit=100&year1=1900&year2=1970&format=Vinyl&price=over40&genre=Jazz&currency=USD&style=Hard+Bop&page="
# # not just bebop
# _url = "https://pitchfork.com/reviews/albums/?genre=jazz/"
# url= _url + str(pagestart) 

# # starting url
# driver.get(url)


'''
This function parses the HTML of the page and attempts to gather attributes like
artist name, album, genre, date, and the review text itself, instead inputting a
null value if the requested element is not found on the page. All of the data are
put into a Pandas dataframe and returned for use in the gather function.
VARIABLES
album_link - A string that refers to the album section of a link to a Pitchfork 
review.
e.g. '/reviews/albums/neil-young-promise-of-the-real-visitor/'
'''
def gather_info(album_link):
    #page = requests.get("https://pitchfork.com" + album_link) #request URL
    page = requests.get("https://pitchfork.com" + album_link) #request URL
    soup = BeautifulSoup(page.content, 'html.parser') #parse with beautifulsoup
    title = str(soup.find('title').string) #album and artist 
    try:
        score = float((soup.find(class_="score").string)) #score
    except AttributeError:
        score = None
    try:
        genre = soup.find(class_="genre-list__link").string #genre
    except AttributeError:
        genre = None
    sents = [element.text for element in soup.find_all('p')] #cleaned text output
    string = " ".join(sents)
 
    # test
    try:
        othercat= [div.text for div in soup.find_all('a')]
        #print(othercat)
    except AttributeError:
        othercat=None
 
    # get img
    #img1=soup.find('img')
    try:
        img= [item['src'] for item in soup.find_all('img')]
        #print('bam', img)
    except AttributeError:
        img = None
    
    # get audio
    
    try:    
        
        #audio = soup.find(class_="multi-link").string #genre
        #audio= [item['href'] for item in soup.find_all('multi-link')]
        
#         for li in soup.find_all(class_="multi-link"):
#             audio.append(li.get('href'))
    except AttributeError:
        audio= None
    
    
    try:
        date = str(soup.find(class_="pub-date").string) #date
    except AttributeError:
        date = None
    #create dataframe with column labels
    d = {'artist': [get_artist(title)], 'album': [get_album(title)], 'score': [score], 'genre': [genre], 'review': [string], 'othercat':[othercat], 'img':[img], 'audio':[audio], 'best': [1 if "Best new" in string else 0], 'date': [date]}
    df = pd.DataFrame(data=d)
    return df


In [195]:
'''
This function starts at Pitchfork's album reviews page and searches through a
requested number of pages from a given start page, adding each album link to the
queue to be scraped by the gather_info function and returning them in a list.
VARIABLES
pages, startPage - Integers that refer to the number of pages to scrape and the
page to start on, respectively, while scraping through Pitchfork's album reviews
page.
. 
'''
def gather_links(pages, startPage):
    pageList = [] #list of album review pages
    linkList = [] #list of album links
    
    for x in range(startPage,(startPage+pages)): #check the first n pages after the requested one
        
        pageList.append(requests.get("https://pitchfork.com/reviews/albums/?genre=jazz&page=" + str(x))) #add each page to list
        #pageList.append(requests.get("https://pitchfork.com/reviews/albums/?page=" + str(x))) #add each page to list
        #print('x',x,'pagelist',pageList)
    for page in pageList:
        print('page',page)
        soup = BeautifulSoup(page.content, 'html.parser') #parse its contents
        #print(soup)
        links = soup.find_all(class_="review__link") #gather its links (in raw html)
        #print('links',links)
        for link in links: #for each link
            linkList.append(link.get('href')) #append only the link itself
        
    return linkList


In [196]:
'''
This function retreives the artist name from the scraped title string.
VARIABLES
title - A string of a cleaned Pitchfork album review title.
'''
def get_artist(title):
    str = ''
    for character in title: #for each character in title
        #add to string up until ':' 
        #if character is not ':':
        if character != ':':
            str += character
        else:
            break
    return str


In [197]:
'''
This function retreives the album name from the scraped title string.
VARIABLES
title - A string of a cleaned Pitchfork album review title.
'''          
def get_album(title):
    str = ''
    #find ':' and index and start there
    index = title.find(":")
    title = title[index+2:]
    #for each character afterwards, add it until '|'
    for character in title:
        #if character is '|':
        if character == '|':
            break
        else:
            str +=character
    return str[:-14] #return just the title



In [198]:
'''
This function uses the other two to gather the requested number of pages starting
from a given page, then adding them all into a single Pandas dataframe, which
is then exported to a CSV file in the given location of the user's desktop.
VARIABLES
pages, startPages - Integers that refer to the number of pages to scrape and the
page to start on, respectively, while scraping through Pitchfork's album reviews
page.
fileLocation - A string that gives a path in the user's desktop where the data
should be saved. 
e.g. 'C:/Users/Evan/Documents'
fileName - A string that gives the desired name of the .csv file. 
e.g. 'p4kReview'
'''
def gather(pages, startPage, fileLocation, fileName):
    linkList = gather_links(pages, startPage) #gather links
    print(linkList)
    first = True #special variable for first scrape
    newDF = pd.DataFrame()
#     global data
#     data=0
    for link in linkList: #for each link        
        
        data = gather_info(link) #gather info
        #if first, newDF becomes the initial dataframe
        if first:
            newDF = data
            first = False
        #otherwise append it
        else:
            newDF = newDF.append(data, ignore_index = True)
    #when scraping complete, export to .csv 
    newDF.to_csv(path_or_buf = fileLocation + "/" + fileName + ".csv")
    #return true if gather was successful
    return True

In [199]:
gather(1,1,'/Users/chrischan/Documents/02_Class/Metis/bootcamp/githubrepo/Metis_proj5/data','pitchfork_jazz_audio')

page <Response [200]>
['/reviews/albums/bobby-womack-the-poet-the-poet-ii/', '/reviews/albums/wau-wau-collectif-yaral-sa-doom/', '/reviews/albums/various-artists-j-jazz-volume-3-deep-modern-jazz-from-japan/', '/reviews/albums/pino-palladino-blake-mills-notes-with-attachments/', '/reviews/albums/sam-gendel-fresh-bread/', '/reviews/albums/femi-kuti-made-kuti-legacy/', '/reviews/albums/archie-shepp-jason-moran-let-my-people-go/', '/reviews/albums/patricia-brennan-maquishti/', '/reviews/albums/baldi-gerycz-duo-after-commodore-perry-service-plaza/', '/reviews/albums/roland-haynes-second-wave/', '/reviews/albums/winston-cw-good-guess/', '/reviews/albums/jahari-massamba-unit-pardon-my-french/']


True